In [2]:
import numpy as np
import pandas as pd

In [89]:
clean = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/clean.csv")

In [90]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [91]:
clean  = fun.delete_id_columns(clean) 
clean = fun.replace_NAN_with_na(clean)
clean = fun.entry_to_lowercase(clean)
clean = fun.remove_underscores_spaces(clean)
clean = fun.impute_data(clean)
clean = fun.convert_to_categorical(clean)
clean

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,YEAR,Country,HHsizemembers,HHsizeMAE,HouseholdType,Head_EducationLevel,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,2.0,groundnut,na,na,2015,tanzania,4,3.32,single,primary,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,2.0,millet,na,na,2015,tanzania,10,5.85,together,primary,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,2.0,groundnut,na,na,2015,tanzania,5,4.32,together,illiterate,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,2.0,cowpea,na,na,2015,tanzania,11,9.33,together,primary,...,0.000000,0.000000,1.000000,0.322767,257.234727,257.234727,0,1.000000,0.000000,africa
4,2.0,sesame,na,na,2015,tanzania,6,5.41,together,secondary,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12188,3.0,maize,goodharvest,monoculture,2018,ghana,8,5.43,couple,primary,...,655.292919,0.000000,1.000000,0.779661,1615.869392,339.743848,7,0.629176,0.370824,africa
12189,5.0,maize,goodharvest,monoculture,2018,ghana,7,4.78,couple,noschool,...,0.000000,0.000000,1.000000,0.322767,355.648536,355.648536,5,1.000000,0.000000,africa
12190,2.0,maize,badharvest,intercrop,2018,ghana,4,3.20,mansingle,noschool,...,49.859244,0.000000,0.456522,0.456522,379.050204,276.284247,0,0.000000,1.000000,africa
12191,2.0,maize,badharvest,intercrop,2018,ghana,4,3.10,couple,noschool,...,0.000000,0.000000,1.000000,0.322767,693.504198,693.504198,0,1.000000,0.000000,africa


In [92]:
clean.dtypes

crop_count                                       float64
crop_name_1                                     category
crop_harvest_1                                  category
crop_intercrop_1                                category
YEAR                                               int64
Country                                         category
HHsizemembers                                      int64
HHsizeMAE                                        float64
HouseholdType                                   category
Head_EducationLevel                             category
LandOwned                                        float64
LandCultivated                                   float64
LivestockHoldings                                float64
NrofMonthsFoodInsecure                             int64
PPI_Threshold                                    float64
PPI_Likelihood                                   float64
HFIAS_status                                    category
score_HDDS_GoodSeason          

In [93]:
market_data, pred_market = fun.drop_response_rows_with_NAs(clean, "Market_Orientation", "PPI_Likelihood") #2


In [94]:
# market_data = fun.delete_id_columns(clean) #1
# market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation", "PPI_Likelihood") #2
# market_data = fun.replace_NAN_with_na(market_data) #3
# market_data = fun.entry_to_lowercase(market_data) #4
# market_data = fun.remove_underscores_spaces(market_data) #5
# market_data = fun.convert_to_categorical(market_data) #6
# market_data = fun.impute_data(market_data)

In [95]:

market_data.YEAR = market_data.YEAR.astype('category')

In [96]:
def standarize_data(df):
    
    """
    Input: a dataset
    action: returns numeric column values scaled by mean and standard deviation
    """
    numeric_data = df.select_dtypes(include=['float64', 'int64'])
    for i in numeric_data.columns:
        df[i] = (df[i] - df[i].mean())/df[i].std()
    return df

In [97]:
market_data = standarize_data(market_data)

In [98]:
market_data.dtypes

crop_count                                       float64
crop_name_1                                     category
crop_harvest_1                                  category
crop_intercrop_1                                category
YEAR                                            category
Country                                         category
HHsizemembers                                    float64
HHsizeMAE                                        float64
HouseholdType                                   category
Head_EducationLevel                             category
LandOwned                                        float64
LandCultivated                                   float64
LivestockHoldings                                float64
NrofMonthsFoodInsecure                           float64
HFIAS_status                                    category
score_HDDS_GoodSeason                            float64
score_HDDS_farmbasedGoodSeason                   float64
score_HDDS_purchasedGoodSeason 

## Ridge Regression y = Market orientation

In [99]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### 1. create dummy

In [100]:
def get_dummyXs_y(df, y_var):
    
    y = df[y_var]
    X  = df.drop(y_var, axis = 1)
    X_cat = X.select_dtypes(include = ["category", "O"])
    X_num = X.select_dtypes(include=['float64', 'int64'])
    
    X_cat_dummy = pd.get_dummies(X_cat)
    newX = pd.concat([X_num, X_cat_dummy], axis = 1)
    
    return newX, y

In [101]:
X_market_ridge = get_dummyXs_y(market_data, "Market_Orientation")[0]
y_market_ridge = get_dummyXs_y(market_data, "Market_Orientation")[1]

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X_market_ridge,y_market_ridge, test_size = 0.3, random_state = 2021)


In [103]:
#ridge_cv=RidgeCV(alphas=np.arange(2,,0.001))
ridge_cv=RidgeCV(alphas=np.arange(0.01,2,0.001))
model_cv=ridge_cv.fit(X_train,y_train)
model_cv.alpha_

1.9989999999999983

In [104]:
#refit model with alpha =3 and get score
rir = Ridge(alpha = model_cv.alpha_)
rir.fit(X_train,y_train)
y_pred_ridge = rir.predict(X_test)
MSE_ridge_market = mean_squared_error(y_test,y_pred_ridge)
MSE_ridge_market

3.824210872472297

In [86]:
X_train.columns

Index(['crop_count', 'YEAR', 'HHsizemembers', 'HHsizeMAE', 'LandOwned',
       'LandCultivated', 'LivestockHoldings', 'NrofMonthsFoodInsecure',
       'score_HDDS_GoodSeason', 'score_HDDS_farmbasedGoodSeason',
       ...
       'Head_EducationLevel_religiousschool', 'Head_EducationLevel_secondary',
       'HFIAS_status_foodsecure', 'HFIAS_status_mildlyfi',
       'HFIAS_status_moderatelyfi', 'HFIAS_status_severelyfi',
       'continent_africa', 'continent_asia', 'continent_centralamerica',
       'continent_southamerica'],
      dtype='object', length=179)

In [ ]:
# -2.17617050e-02,  1.57421573e+00, -1.55559476e+00, -4.08594181e-03

In [105]:
rir.coef_

array([ 4.75702180e-04, -1.12790058e-02, -1.66693816e-02, -6.05689323e-03,
       -1.09441019e-03, -1.22792477e-01, -8.57191904e-03, -9.26553924e-03,
       -4.44942265e-03,  6.24363458e-03, -1.86112294e-02, -2.89572604e-02,
       -8.33169927e-03,  1.82747039e-05, -1.31277188e-01, -2.37528617e-01,
        2.90967422e-02,  2.38064982e-02,  2.24617599e-02,  2.03904027e-02,
       -1.44294329e-01,  8.45626746e-01,  1.79935495e-01, -1.85672677e-01,
       -2.39395898e-01,  4.20047202e-02, -3.99978321e-02,  6.04090207e-03,
       -5.16819326e-02,  5.16819326e-02,  0.00000000e+00, -6.93219381e-01,
        0.00000000e+00,  1.15671958e-01, -6.63708067e-02, -1.08870058e-02,
        1.69179128e-02,  6.85237157e-01, -3.15519395e-01,  4.51044720e-02,
       -4.09767783e-02,  1.48234684e-01,  1.35150234e-01,  9.16040860e-02,
       -3.01026516e-01,  2.68603015e-01, -1.35467287e-01,  1.75205467e-02,
        1.89632355e-01,  3.05539086e-01, -8.03181326e-02,  1.95763138e-01,
        9.93754867e-03, -

In [106]:
feature_imp  = pd.DataFrame([X_train.columns, rir.coef_]).T

In [107]:
feature_imp = feature_imp.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp.iloc[0:15,:]

,0,1
0,livestock_prodsales_USD_PPP_pHH_Yr,0.845627
1,crop_name_1_broadbeans,0.685237
2,Country_india,0.51684
3,Head_EducationLevel_none,0.480222
4,HouseholdType_nonparenthead,0.447677
5,Country_zambia,0.430223
6,Head_EducationLevel_islamicschool,0.387211
7,crop_name_1_mango,0.383287
8,Country_mali,0.383044
9,HouseholdType_together,0.380135


## Ridge regression with y=PPI_Likelihood

In [73]:
clean = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/clean.csv")

In [74]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [75]:
clean  = fun.delete_id_columns(clean) 
clean = fun.replace_NAN_with_na(clean)
clean = fun.entry_to_lowercase(clean)
clean = fun.remove_underscores_spaces(clean)
clean = fun.impute_data(clean)
clean = fun.convert_to_categorical(clean)
clean

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,YEAR,Country,HHsizemembers,HHsizeMAE,HouseholdType,Head_EducationLevel,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,2.0,groundnut,na,na,2015,tanzania,4,3.32,single,primary,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,2.0,millet,na,na,2015,tanzania,10,5.85,together,primary,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,2.0,groundnut,na,na,2015,tanzania,5,4.32,together,illiterate,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,2.0,cowpea,na,na,2015,tanzania,11,9.33,together,primary,...,0.000000,0.000000,1.000000,0.317386,257.234727,257.234727,0,0.999885,0.000115,africa
4,2.0,sesame,na,na,2015,tanzania,6,5.41,together,secondary,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13305,3.0,maize,goodharvest,monoculture,2018,ghana,8,5.43,couple,primary,...,655.292919,0.000000,1.000000,0.779661,1615.869392,339.743848,7,0.629176,0.370824,africa
13306,5.0,maize,goodharvest,monoculture,2018,ghana,7,4.78,couple,noschool,...,0.000000,0.000000,1.000000,0.317386,355.648536,355.648536,5,1.000000,0.000000,africa
13307,2.0,maize,badharvest,intercrop,2018,ghana,4,3.20,mansingle,noschool,...,49.859244,0.000000,0.456522,0.456522,379.050204,276.284247,0,0.000000,1.000000,africa
13308,2.0,maize,badharvest,intercrop,2018,ghana,4,3.10,couple,noschool,...,0.000000,0.000000,1.000000,0.317386,693.504198,693.504198,0,1.000000,0.000000,africa


In [77]:
ppi_data, pred_market = fun.drop_response_rows_with_NAs(clean,"PPI_Likelihood" ,"Market_Orientation") #2


In [153]:
# ppi_data = fun.delete_id_columns(clean) #1
# ppi_data, pred_market = fun.drop_response_rows_with_NAs(ppi_data,"PPI_Likelihood" ,"Market_Orientation") #2
# ppi_data = fun.replace_NAN_with_na(ppi_data) #3
# ppi_data = fun.entry_to_lowercase(ppi_data) #4
# ppi_data = fun.remove_underscores_spaces(ppi_data) #5
# ppi_data = fun.convert_to_categorical(ppi_data) #6
# ppi_data = fun.impute_data(ppi_data)#7

In [78]:
ppi_data.YEAR = ppi_data.YEAR.astype('category')
ppi_data = standarize_data(ppi_data)

In [79]:
X_ppi = get_dummyXs_y(ppi_data, "PPI_Likelihood")[0]
y_ppi = get_dummyXs_y(ppi_data, "PPI_Likelihood")[1]

In [80]:
X_train_ppi, X_test_ppi, y_train_ppi, y_test_ppi = train_test_split(X_ppi ,y_ppi, test_size = 0.3, random_state = 2021)


In [81]:
ppi_ridge_cv=RidgeCV(alphas=np.arange(0.01,2,0.001))
ppi_model_cv=ppi_ridge_cv.fit(X_train_ppi,y_train_ppi)
ppi_model_cv.alpha_

0.01

In [82]:
#refit model with alpha =3 and get score
ridge_ppi = Ridge(alpha = ppi_model_cv.alpha_)
ridge_ppi.fit(X_train_ppi,y_train_ppi)
y_pred_ridge_ppi = ridge_ppi.predict(X_test_ppi)
MSE_ppi = np.mean((y_pred_ridge_ppi-np.array(y_test_ppi))**2)
MSE_ppi

2.35103808029831

In [86]:
ridge_ppi.coef_

array([-1.85618711e-02,  1.01714304e+00, -6.19939509e-01, -3.56559502e-03,
        1.24300050e-03,  4.57410737e-03,  8.95323896e-02, -7.31731886e-02,
       -3.63824114e-03, -1.02422019e-02, -5.82050791e-02, -6.89129338e-03,
        2.58434348e-02, -9.25753424e-02, -2.56197776e-01, -4.56263965e-01,
        5.01442407e-02,  5.00874232e-02,  5.01056313e-02,  5.01131638e-02,
        8.71759685e-04,  5.15351836e-03, -8.69190055e-04, -3.29040766e-03,
       -4.90424169e-02,  1.11734487e+00, -1.11044333e+00,  2.82550942e-02,
        7.64826080e-03, -7.64826071e-03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  2.02624957e-01,  3.43743700e-02,  1.55758296e-01,
       -1.21766729e-01,  4.00748134e-02,  6.91594257e-01,  1.33769168e-01,
       -8.50707831e-02, -8.88707634e-02,  1.56932504e+00, -1.58515277e-01,
       -4.77463707e-01,  1.34017319e-02, -4.43228363e-01,  2.96500357e-02,
        2.81283352e-02,  4.26198362e-01,  4.67306234e-01, -2.16428014e-01,
        0.00000000e+00,  

In [87]:
feature_imp_ppi  = pd.DataFrame([X_train.columns, ridge_ppi.coef_]).T
feature_imp_ppi = feature_imp_ppi.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp_ppi.iloc[0:15,:]

,0,1
0,crop_name_1_chickpeas,1.56933
1,Country_zambia,1.52008
2,Country_mali,1.46559
3,Country_elsalvador,1.16155
4,Food_Availability_kCal_MAE_day,1.11734
5,Country_malawi,1.08616
6,HHsizemembers,1.01714
7,Head_EducationLevel_1(�colerurale),0.900755
8,Head_EducationLevel_5alphab�tisation,0.711817
9,crop_name_1_cabbage,0.691594


In [88]:
pd.concat([feature_imp_ppi, feature_imp],axis = 1).iloc[0:20,:]

,0,1,0,1
0,crop_name_1_chickpeas,1.56933,crop_name_1_broadbeans,0.763437
1,Country_zambia,1.52008,Head_EducationLevel_none,0.496279
2,Country_mali,1.46559,HouseholdType_nonparenthead,0.474764
3,Country_elsalvador,1.16155,crop_name_1_ognons,0.469478
4,Food_Availability_kCal_MAE_day,1.11734,Head_EducationLevel_islamicschool,0.44732
5,Country_malawi,1.08616,Country_zambia,0.421361
6,HHsizemembers,1.01714,Country_mali,0.411169
7,Head_EducationLevel_1(�colerurale),0.900755,Country_india,0.410221
8,Head_EducationLevel_5alphab�tisation,0.711817,crop_name_1_tomato,0.350568
9,crop_name_1_cabbage,0.691594,crop_name_1_oranges,0.345062


In [85]:
clean.Head_EducationLevel.unique()

[primary, illiterate, secondary, literate, noschool, ..., postsecondary, informal, none, religiousschool, islamicschool]
Length: 27
Categories (27, object): [primary, illiterate, secondary, literate, ..., informal, none, religiousschool, islamicschool]

In [172]:
clean.Head_EducationLevel.isna().sum()

412

In [ ]:
search_values = ['primary', 'illiterate', 'secondary', 'literate', 'no_school',]

In [ ]:
def drop_head_edu_rows(df, x):
    condition = df[x]
    rows_to_delete = df[np.isnan(condition)].index
    
    #save all rows with y=NA. Will use this data to predict the y later.
        prediction_dataset = df.iloc[rows_to_delete,:]
    
    #create new data
        new_data = df.drop(labels=rows_to_delete, axis=0)
        new_data = new_data.reset_index(drop=True)

In [170]:
condition = clean.Head_EducationLevel
rows =  clean[(condition)].index



ValueError: cannot mask with array containing NA / NaN values

In [171]:
clean[clean.Head_EducationLevel.str.contains('primary', "illiterate",case=False)]

TypeError: contains() got multiple values for argument 'case'

In [169]:
clean.Head_EducationLevel[clean.Head_EducationLevel == "'primary', 'illiterate', 'secondary', 'literate', 'no_school', nan")]



SyntaxError: invalid syntax (<ipython-input-169-580f3d44ef86>, line 1)